In [1]:
# All files are here
import os
import numpy as np
from PIL import Image
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from tqdm import tqdm

In [6]:
# We write a function to convert the images folder to .dat type
def images_to_dat(images_folder, output_file):
    with open(output_file, "wb") as dat_file:
        for filename in sorted([f for f in os.listdir(images_folder) if f != "output.csv" and f.endswith(".jpeg")], key = lambda x: int(x.split('_')[0])):
        # for filename in os.listdir(images_folder):
            if filename.endswith(".jpeg"):
            # Read the image file
                img_path = os.path.join(images_folder, filename)
                img = cv2.imread(img_path)
                if img is None:
                    print(f"Failed to load {filename}")
                    continue 
                img = img > 32
                img = img.astype(np.uint8) * 255
                img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
                img_array = np.array(img)

                # Flatten the image and convert to bytes, for easier storage
                image_bytes = img_array.flatten().tobytes()

                # Write the image data and store that in .dat file
                dat_file.write(image_bytes)

In [7]:
# Use the function
images_to_dat("training_data", "training_data.dat")
images_to_dat("validation_data", "validation_data.dat")

In [9]:
# We then create an array of images
# This is a breakthrough, at least for me, since I cannot do anything to store a large amount like that
# to a list, until I find np.memmap
training_data = []
training_data.append(np.memmap("training_data.dat", dtype = np.uint8, mode = 'r').reshape(
                        (-1, 64, 60))) # since it is 60 x 60 images
training_data = np.concatenate(training_data)
print(training_data.shape)

(287102, 64, 60)


In [10]:
# Similarly, for validation_data
validation_data = []
validation_data.append(np.memmap("validation_data.dat", dtype = np.uint8, mode = 'r').reshape(
    (-1, 64, 60)))
validation_data = np.concatenate(validation_data)
print(validation_data.shape)

(123045, 64, 60)
